# Notebook ETL_user_reviews

Este notebook tiene como objetivo realizar las tareas de Extracción, Transformación y Carga (ETL) de los datos del archivo user_reviews.json.gz.

## 1. Importamos librerías

In [1]:
# Importamos pandas para la manipulación y análisis de datos tabulares
import pandas as pd

# Importamos numpy para operaciones numéricas eficientes y manipulación de matrices
import numpy as np

# Importamos gzip para trabajar con archivos comprimidos en formato gzip
import gzip

# Importamos json para trabajar con datos en formato JSON
import json

# Importamos ast (Abstract Syntax Trees) para análisis sintáctico y evaluación de expresiones Python
import ast

# Importamos os para interactuar con el sistema operativo y manipular archivos y directorios
import os

# Importamos langdetect para detectar automáticamente el idioma de los textos
from langdetect import detect

# Importamos la biblioteca re para el manejo de expresiones regulares
import re

# Importamos NLTK para procesamiento de lenguaje natural
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Descargamos los recursos necesarios de NLTK
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# %load_ext autoreload y %autoreload 2 se utilizan para recargar automáticamente los módulos importados
# Esto es útil cuando estamos desarrollando y queremos que los cambios realizados en un módulo se reflejen automáticamente
%load_ext autoreload
%autoreload 2

# Importamos warnings para controlar las advertencias que pueden surgir durante la ejecución del código
import warnings
warnings.filterwarnings("ignore")  # Ignoramos las advertencias para mantener el flujo de trabajo limpio


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Ahora continuamos con los pasos del proceso ETL.

## 2. Extracción (carga) de los conjuntos de datos

En esta etapa del proceso ETL, se realiza la extracción de datos desde el archivo comprimido en formato *gzip* llamado **user_reviews.json.gz**. Este archivo contiene reseñas de usuarios sobre diferentes juegos en la plataforma Steam.

Archivo origen: **user_reviews.json.gz** (Ubicado en *Data\Raw\user_reviews.json.gz*)

In [2]:
# Ruta al archivo gzip comprimido
ruta_review = '../Data/Raw/user_reviews.json.gz'

# Lista para almacenar las filas de datos descomprimidos
filas_review = []

# Se lee cada línea del archivo gzip y se descomprime
with gzip.open(ruta_review, 'rt', encoding='utf-8') as f:
    for line in f.readlines():
        # Se utiliza ast.literal_eval para evaluar la cadena como una expresión Python segura
        # Esto convierte la cadena JSON en un diccionario Python
        filas_review.append(ast.literal_eval(line))

# Se convierte la lista de diccionarios en un DataFrame de pandas
df_reviews = pd.DataFrame(filas_review)


## 3. Explorar el conjunto de datos

En la sección de "Exploración del conjunto de datos" de las reseñas de usuarios (user_reviews), se inicia con una visualización de las primeras filas del DataFrame mediante el método head(), lo que proporciona una vista previa de los datos y ayuda a entender su estructura y contenido inicial. Posteriormente, se imprime información general del DataFrame, utilizando el método info(), que muestra detalles como el número de entradas no nulas por columna, el tipo de datos y la memoria utilizada. Esta exploración inicial permite obtener una visión panorámica de la composición y la calidad de los datos, lo que facilita la identificación de posibles áreas de interés y la planificación de pasos adicionales de preprocesamiento y análisis.

In [3]:
# Se muestra el DataFrame resultante
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Procedemos a desanidar los elementos de la columna "reviews"

In [5]:
# Normalizar la columna 'items' y expandirla en nuevas columnas
df_reviews = pd.json_normalize(df_reviews.to_dict('records'), record_path='reviews', meta=['user_id', 'user_url'])


In [6]:
df_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


In [7]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59305 non-null  object
 7   user_id      59305 non-null  object
 8   user_url     59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


Vemos que a pesar de que hay elementros vacíos en el dataset, esto no se refleja en el conteo de no nulos, se deberá proceder a convertir los strings vacíos y otros elementos similares a valores nulos (nan)

## 4. Limpiar y preprocesar el conjunto de datos

En la etapa de limpieza y preparación de datos, el código proporcionado reemplaza valores vacíos, 'null' y 'None' con NaN en todo el DataFrame de reseñas de usuarios. Esta acción uniformiza la representación de los valores faltantes, facilitando su manejo y análisis subsiguiente.

In [8]:
# Reemplazar valores vacíos, 'null' y 'None' con NaN en todo el DataFrame
df_reviews.replace(['', 'null', 'None'], np.nan, inplace=True)

In [9]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        8151 non-null   object
 1   posted       59305 non-null  object
 2   last_edited  6140 non-null   object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59275 non-null  object
 7   user_id      59305 non-null  object
 8   user_url     59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


Evaluaremos qué columnas tienen más de un 80% de valores nulos y podrían serun problema para el análisis

In [10]:
# Calcular el porcentaje de valores nulos por columna
porcentaje_nulos = df_reviews.isnull().mean() * 100

# Filtrar las columnas que tienen un porcentaje de nulos mayor al umbral (80%)
columnas_con_nulos = porcentaje_nulos[porcentaje_nulos > 80]

# Mostrar las columnas con un alto porcentaje de nulos
print("Columnas con más del 80% de valores nulos:")
print(columnas_con_nulos)


Columnas con más del 80% de valores nulos:
funny          86.255796
last_edited    89.646741
dtype: float64


Dado que estas columnas no son indespensables para el proyecto, se procederá a su eliminación

In [11]:
# Obtener el nombre de las columnas con más del 80% de valores nulos
columnas_a_eliminar = columnas_con_nulos.index.tolist()

# Eliminar las columnas del DataFrame df_reviews
df_reviews = df_reviews.drop(columns=columnas_a_eliminar)


In [12]:
df_reviews.head()

,posted,item_id,helpful,recommend,review,user_id,user_url
0,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


También eliminamos otras columnas con información irrelevante para el proyecto

In [13]:
# Eliminar las columnas 'helpful' y 'user_url' del DataFrame df_reviews
df_reviews = df_reviews.drop(columns=['helpful', 'user_url'])

In [14]:
df_reviews.head()

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637


Ahora procedemos a evaluar la cantidad de filas duplicadas

In [15]:
# Encuentra registros duplicados basados en las columnas especificadas
filas_duplicadas = df_reviews[df_reviews.duplicated(subset=['user_id', 'item_id', 'posted', 'review'], keep=False)]

# Muestra el recuento de filas duplicadas
print("Número de filas duplicadas:", len(filas_duplicadas))


Número de filas duplicadas: 1736


Procedemos a eliminar los duplicados

In [16]:
print("Número de filas antes de eliminar duplicados:", len(df_reviews))

# Eliminar las filas duplicadas basadas en las columnas especificadas
df_reviews = df_reviews.drop_duplicates(subset=['user_id', 'item_id', 'posted', 'review'], keep='first')

# Verificar la forma del DataFrame después de eliminar duplicados
print("Número de filas después de eliminar duplicados:", len(df_reviews))


Número de filas antes de eliminar duplicados: 59305
Número de filas después de eliminar duplicados: 58431


Ahora queremos obtener el dato del año de la columna "posted", para lo cual primero observaremos algunos de sus valores únicos

In [17]:
# Obtener una lista de los valores únicos en la columna 'posted'
valores_unicos_posted = df_reviews['posted'].unique()

# Mostrar la lista de valores únicos de 'posted'
print("Valores únicos de 'posted':")
print(valores_unicos_posted)


Valores únicos de 'posted':
['Posted November 5, 2011.' 'Posted July 15, 2011.'
 'Posted April 21, 2011.' ... 'Posted February 18, 2013.'
 'Posted November 13, 2012.' 'Posted November 3, 2012.']


Parece que todos los valores empiezan por 'Posted ', primero corroboraremos dicha afirmación

In [18]:
# Verificar si todos los valores de la columna 'posted' comienzan con 'Posted '
todos_empiezan_con_posted = df_reviews['posted'].str.startswith('Posted ').all()

# Imprimir el resultado
print("¿Todos los valores de la columna 'posted' empiezan con 'Posted '?", todos_empiezan_con_posted)


¿Todos los valores de la columna 'posted' empiezan con 'Posted '? True


Dado que sí es verdad, procederemos a eliminar la cadena 'Posted ' para poder trabajar con las fechas de la columna posted

In [19]:
# Eliminar la cadena 'Posted ' de todos los valores en la columna 'posted'
df_reviews['posted'] = df_reviews['posted'].str.replace('Posted ', '')

# Verificar que la cadena 'Posted ' haya sido eliminada correctamente
todos_empiezan_con_posted = df_reviews['posted'].str.startswith('Posted ').any()
print("¿Todos los valores de la columna 'posted' comienzan con 'Posted '?", todos_empiezan_con_posted)


¿Todos los valores de la columna 'posted' comienzan con 'Posted '? False


Ahora verificamos si todos los valores terminan con '.'

In [20]:
# Verificar si todos los valores de la columna 'posted' terminan con '.'
todos_terminan_con_punto = df_reviews['posted'].str.endswith('.').all()

# Imprimir el resultado
print("¿Todos los valores de la columna 'posted' terminan con '.'?", todos_terminan_con_punto)


¿Todos los valores de la columna 'posted' terminan con '.'? True


Dado que sí es verdad, procederemos a eliminar el caracter '.' para poder trabajar con las fechas de la columna posted

In [21]:
# Eliminar la cadena '.' al final de todos los valores en la columna 'posted'
df_reviews['posted'] = df_reviews['posted'].str.rstrip('.')

# Verificar que la cadena '.' haya sido eliminada correctamente
todos_terminan_con_punto = df_reviews['posted'].str.endswith('.').any()
print("¿Todos los valores de la columna 'posted' terminan con '.'?", todos_terminan_con_punto)


¿Todos los valores de la columna 'posted' terminan con '.'? False


Ahora podemos verificar que los valores de posted serán más sencillos de transformar a tipo de dato datetime

In [22]:
# Obtener una lista de los valores únicos en la columna 'posted'
valores_unicos_posted = df_reviews['posted'].unique()

# Mostrar la lista de valores únicos de 'posted'
print("Valores únicos de 'posted':")
print(valores_unicos_posted)

Valores únicos de 'posted':
['November 5, 2011' 'July 15, 2011' 'April 21, 2011' ...
 'February 18, 2013' 'November 13, 2012' 'November 3, 2012']


Intentaremos convertir cada fila a formato fecha, la razón de no usar un solo patrón específico

In [23]:
# Definir una función para convertir valores a fecha de manera dinámica
def convertir_a_fecha(valor):
    try:
        fecha = pd.to_datetime(valor)
        return fecha
    except:
        return pd.NaT

# Aplicar la función a la columna 'posted'
df_reviews['posted'] = df_reviews['posted'].apply(lambda x: convertir_a_fecha(x))

# Verificar la conversión
print(df_reviews['posted'].head())


0   2011-11-05
1   2011-07-15
2   2011-04-21
3   2014-06-24
4   2013-09-08
Name: posted, dtype: datetime64[ns]


Ahora verificaremos la cantidad de fechas que logró registrar

In [24]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   posted     48498 non-null  datetime64[ns]
 1   item_id    58431 non-null  object        
 2   recommend  58431 non-null  bool          
 3   review     58401 non-null  object        
 4   user_id    58431 non-null  object        
dtypes: bool(1), datetime64[ns](1), object(3)
memory usage: 2.3+ MB


Después crearemos la columna 'year' extrayendo el año de 'posted'

In [25]:
# Convertir la columna 'posted' al formato de fecha si aún no lo está
df_reviews['posted'] = pd.to_datetime(df_reviews['posted'], errors='coerce')

# Extraer el año de la columna 'posted' y crear la columna 'year'
df_reviews['year'] = df_reviews['posted'].dt.year

# Verificar la nueva columna 'year'
print(df_reviews[['posted', 'year']].head())


      posted    year
0 2011-11-05  2011.0
1 2011-07-15  2011.0
2 2011-04-21  2011.0
3 2014-06-24  2014.0
4 2013-09-08  2013.0


Dado que faltan muchos valores nulos, procedemos a rellenar los valores nulos en la columna 'year' mediante interpolación lineal por grupo ('item_id'), primero debemos ordenar el DataFrame por 'item_id' y 'year' para asegurar que la interpolación se realice correctamente. Luego, podemos aplicar la interpolación por grupo utilizando el método groupby() de Pandas junto con interpolate().

In [26]:
# Ordenar el DataFrame por 'item_id' y 'year'
df_reviews.sort_values(['item_id', 'year'], inplace=True)

# Rellenar valores nulos en 'year' mediante interpolación lineal
df_reviews['year'] = df_reviews['year'].interpolate(method='linear')

# Redondear los valores de la columna 'year' para que sean enteros
df_reviews['year'] = df_reviews['year'].astype(int)

# Verificar los cambios
df_reviews[['item_id', 'year']].head()


,item_id,year
5331,10,2011
22702,10,2012
35539,10,2012
43134,10,2012
24137,10,2013


In [27]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 5331 to 32632
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   posted     48498 non-null  datetime64[ns]
 1   item_id    58431 non-null  object        
 2   recommend  58431 non-null  bool          
 3   review     58401 non-null  object        
 4   user_id    58431 non-null  object        
 5   year       58431 non-null  int32         
dtypes: bool(1), datetime64[ns](1), int32(1), object(3)
memory usage: 2.5+ MB


Gracias a esto ya no hay más valores nulos en 'year' y podemos proceder a eliminar la columna 'posted'

In [28]:
# Eliminar la columna 'posted'
df_reviews.drop(columns=['posted'], inplace=True)

Detectamos los idiomas presentes en nuestras revisiones de usuarios para entender mejor nuestra audiencia. Utilizamos un algoritmo de detección de lenguaje para contar y calcular el porcentaje de revisiones en cada idioma. Este análisis nos proporciona una visión diversa de las preferencias lingüísticas de nuestros usuarios.

In [29]:
# Función para detectar el idioma de cada review
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# Aplicar la función detect_language a la columna 'review' y crear una nueva columna 'language'
df_reviews['language'] = df_reviews['review'].apply(detect_language)

# Contar la cantidad de reviews por cada idioma
language_counts = df_reviews['language'].value_counts().reset_index()
language_counts.columns = ['language', 'review_count']

# Calcular el porcentaje de cada idioma en relación al total de reviews
language_counts['percentage'] = (language_counts['review_count'] / len(df_reviews)) * 100

# Ordenar los idiomas por cantidad de reviews en orden descendente
language_counts = language_counts.sort_values(by='review_count', ascending=False)


In [30]:
# Mostrar el DataFrame con los idiomas, el conteo de reviews y el porcentaje
language_counts


,language,review_count,percentage
0,en,45085,77.159385
1,pt,2156,3.689822
2,es,1257,2.151255
3,de,1132,1.937328
4,so,1011,1.730246
5,af,746,1.276720
6,th,709,1.213397
7,unknown,573,0.980644
8,tl,565,0.966952
9,cy,446,0.763293


Consideraremos a los 5 idiomas más representados dado que abarcan más del 80% de los comentarios, redondearemos el procentaje a dos dígitos

In [31]:
# Mapeo de idiomas
language_mapping = {
    'en': 'English',
    'pt': 'Portuguese',
    'es': 'Spanish',
    'de': 'German',
    'so': 'Somali'
}

# Remapear los idiomas directamente en la columna 'language' y eliminar las filas con idiomas que no están en el mapeo
language_counts['language'] = language_counts['language'].map(language_mapping)
language_counts = language_counts[language_counts['language'].notnull()]

# Redondear el porcentaje a 2 dígitos
language_counts['percentage'] = language_counts['percentage'].round(2)


In [32]:
# Mostrar el DataFrame con los idiomas, el conteo de reviews y el porcentaje
language_counts


,language,review_count,percentage
0,English,45085,77.16
1,Portuguese,2156,3.69
2,Spanish,1257,2.15
3,German,1132,1.94
4,Somali,1011,1.73


Después exportamos el dataframe a la ubicación correspondiente

In [33]:
# Definir la ruta de salida del archivo CSV
output_path = '../Data/Processed/Clean_data/language_counts.csv'

# Exportar el DataFrame a un archivo CSV
language_counts.to_csv(output_path, index=False)

Dado que la mayoría de filas de df_reviews están escritas en idioma inglés y para favorecer el análisis y transformaciones en la siguiente sección, se decidió mantener solo a las reviews en inglés en df_reviews 

In [34]:
# Filtrar las filas donde el idioma de la revisión sea inglés ('en')
df_reviews = df_reviews[df_reviews['language'] == 'en']

# Restablecer el índice después de filtrar
df_reviews.reset_index(drop=True, inplace=True)

In [35]:
df_reviews.head()

,item_id,recommend,review,user_id,year,language
0,10,True,this game is the 1# online action game is awes...,76561198040188061,2011,en
1,10,True,The OG to CS:GO.,epic_doom,2013,en
2,10,True,THE BEST FPS GAME!!!!!,mayshowganmore,2014,en
3,10,True,One of the best childhood games i have ever pl...,BestinTheWorldThund3r,2014,en
4,10,True,People still play this! Siq game,76561198072207162,2014,en


In [36]:
# Eliminar la columna 'language'
df_reviews.drop(columns=['language'], inplace=True)


In [37]:
df_reviews.head()


,item_id,recommend,review,user_id,year
0,10,True,this game is the 1# online action game is awes...,76561198040188061,2011
1,10,True,The OG to CS:GO.,epic_doom,2013
2,10,True,THE BEST FPS GAME!!!!!,mayshowganmore,2014
3,10,True,One of the best childhood games i have ever pl...,BestinTheWorldThund3r,2014
4,10,True,People still play this! Siq game,76561198072207162,2014


## 5. Feature Enginerring

La sección "Feature Engineering" se centra en enriquecer los datos para mejorar el rendimiento de los modelos de machine learning y facilitar el análisis de datos. En esta sección, se aplicará análisis de sentimientos a las reseñas de los usuarios de juegos en Steam. Utilizando la biblioteca NLTK, se asignará a cada reseña un valor numérico que refleje su sentimiento, clasificándolas como negativas, neutrales o positivas. El código correspondiente inicializa un analizador de sentimientos y define una función para asignar valores de sentimiento a cada reseña. Luego, aplica esta función a la columna de reseñas, creando una nueva columna llamada "sentiment_analysis". Finalmente, elimina la columna original de reseñas para simplificar el conjunto de datos.

In [38]:
# Descargar los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

# Inicializar el analizador de sentimientos
sid = SentimentIntensityAnalyzer()

# Función para preprocesar el texto de las reseñas
def preprocess_text(text):
    # Convertir el texto a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenización
    tokens = word_tokenize(text)
    # Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    # Unir tokens en un solo texto
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

# Función para asignar el sentimiento según el análisis de Vader
def get_sentiment(review):
    if pd.isnull(review):
        return 1  # Valor neutral si la reseña está ausente
    else:
        sentiment_score = sid.polarity_scores(review)['compound']
        if sentiment_score >= 0.05:
            return 2  # Valor positivo
        elif sentiment_score <= -0.05:
            return 0  # Valor negativo
        else:
            return 1  # Valor neutral

# Aplicar preprocesamiento de texto a la columna 'review'
df_reviews['processed_review'] = df_reviews['review'].apply(preprocess_text)

# Aplicar el análisis de sentimientos y crear la columna 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['processed_review'].apply(get_sentiment)

# Eliminar las columnas temporales
df_reviews.drop(columns=['review', 'processed_review'], inplace=True)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [39]:
# Mostrar las primeras filas para verificar los cambios
df_reviews.head()

,item_id,recommend,user_id,year,sentiment_analysis
0,10,True,76561198040188061,2011,2
1,10,True,epic_doom,2013,1
2,10,True,mayshowganmore,2014,2
3,10,True,BestinTheWorldThund3r,2014,2
4,10,True,76561198072207162,2014,2


Finalmente realizamos un conteo de cada valor único de la columna sentiment_analysis ('0', '1' y '2'), además de verificar si existe algún valor nulo

In [40]:
# Calcular el conteo y el porcentaje de valores únicos en 'sentiment_analysis'
sentiment_counts = df_reviews['sentiment_analysis'].value_counts()
sentiment_percentages = df_reviews['sentiment_analysis'].value_counts(normalize=True) * 100

# Calcular la cantidad y el porcentaje de valores en blanco o nulos
null_values_count = df_reviews['sentiment_analysis'].isnull().sum()
null_values_percentage = (null_values_count / len(df_reviews)) * 100

# Mostrar los resultados
print("Conteo de valores únicos en 'sentiment_analysis':")
print(sentiment_counts)
print("\nPorcentaje de valores únicos en 'sentiment_analysis':")
print(sentiment_percentages)
print(f"\nCantidad de valores en blanco o nulos en 'sentiment_analysis': {null_values_count}")
print(f"Porcentaje de valores en blanco o nulos en 'sentiment_analysis': {null_values_percentage:.2f}%")


Conteo de valores únicos en 'sentiment_analysis':
sentiment_analysis
2    32237
0     7617
1     5231
Name: count, dtype: int64

Porcentaje de valores únicos en 'sentiment_analysis':
sentiment_analysis
2    71.502717
0    16.894754
1    11.602529
Name: proportion, dtype: float64

Cantidad de valores en blanco o nulos en 'sentiment_analysis': 0
Porcentaje de valores en blanco o nulos en 'sentiment_analysis': 0.00%


## 6. Guardar el conjunto de datos limpio

In [41]:
# Ruta para exportar el archivo CSV
ruta_exportacion = "../Data/Processed/Clean_data/user_reviews_cleaned.csv"

# Exportar el DataFrame a CSV
df_reviews.to_csv(ruta_exportacion, index=False)
